In [ ]:
import numpy as np
from skimage import exposure as ex
import imageio
import cv2
import os
from skimage import metrics

In [ ]:
def he(img):
    if (len(img.shape) == 2):  # gray
        outImg = ex.equalize_hist(img[:, :]) * 255
    elif (len(img.shape) == 3):  # RGB

        # # img.shape :  get dimensions of image (height,width,channels)
        # # height, width, number of channels in image
        # height = img.shape[0]
        # width = img.shape[1]
        # channels = img.shape[2]

        outImg = np.zeros((img.shape[0], img.shape[1], 3))
        for channel in range(img.shape[2]):
            outImg[:, :, channel] = ex.equalize_hist(img[:, :, channel]) * 255

    outImg[outImg > 255] = 255
    outImg[outImg < 0] = 0
    return outImg.astype(np.uint8)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
in_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/raw'
out_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/he_raw'

for imagePath in os.listdir(in_path):
    input_path = os.path.join(in_path, imagePath)
    img = imageio.imread(input_path)
    result = he(img)
    full_out_path = os.path.join(out_path, imagePath)
    cv2.imwrite(full_out_path, result)

In [ ]:
"""

import numpy as np
import cv2
import os

from sklearn.metrics import mean_squared_error

target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/he_raw'

target_data = []
predict_data = []


im_h = 112
im_w = 112
for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 112 x 112
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))
        
    # put the images into array list
    target_data.append(target_img)
    predict_data.append(predicted_img)


# convert arrays into numpy arrays
target_data = np.array(target_data)
predict_data = np.array(predict_data)

print(target_data.shape, predict_data.shape)


%cd '/content/drive/MyDrive/UnderwaterImageEnhancement/models/he-raw-predicted/'

# save the numpy arrays
np.save('target_data', target_data)
np.save('predict_data', predict_data)

"""

In [ ]:
"""

%cd '/content/drive/MyDrive/UnderwaterImageEnhancement/models/he-raw-predicted/'

# load the numpy arrays
target_data = np.load('target_data.npy')
predict_data = np.load('predict_data.npy')

print(target_data.shape, predict_data.shape)

"""

In [ ]:
"""


import numpy as np
  
# Mean Squared Error
MSE = np.square(np.subtract(target_data,predict_data)).mean()
print("MSE for test data :",MSE)

"""

In [ ]:
"""


mse = (np.square(target_data - predict_data)).mean(axis=None)

print("MSE for test data :",mse)


"""

In [ ]:
"""


from math import log10, sqrt

def PSNR(target_data, predict_data):
 mse = np.mean((target_data - predict_data) ** 2)
 print(mse)
 if(mse == 0):
  return 100
 max_pixel = 255.0
 psnr = 20 * log10(max_pixel / sqrt(mse))
 return psnr 


value = PSNR(target_data, predict_data)
print("\nPeak Signal-to-Noise Ratio (PSNR) value is", value, "dB")


"""

In [ ]:
"""

from skimage.metrics import structural_similarity as ssim

ssim_value = ssim(target_data, predict_data, multichannel=True)
print("SSIM:", ssim_value)

"""

In [ ]:
"""

from skimage.measure import compare_ssim

(score, diff) = compare_ssim(target_data, predict_data, full=True, multichannel=True)
diff = (diff * 255).astype("uint8")
print("SSIM:", score)

"""

In [ ]:
###### FULL-REFERENCE IMAGE QUALITY ASSESSMENT IN TERMS OF MSE, PSNR, AND SSIM #######

import numpy as np
import cv2
import os

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity 


target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/he_raw'


im_h = 224
im_w = 224

sumMSE = 0;
sumPSNR = 0;
sumSSIM = 0;
count = 0;


for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 224 x 224
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))


    #calculate the MSE, PSNR, and SSIM 
    MSE = mean_squared_error(target_img, predicted_img)
    PSNR = peak_signal_noise_ratio(target_img, predicted_img)
    SSIM = structural_similarity(target_img, predicted_img, multichannel=True)

    sumMSE = sumMSE + MSE
    sumPSNR = sumPSNR + PSNR
    sumSSIM = sumSSIM + SSIM
    count = count + 1
    
    # print('SSIM: ', SSIM)
    # print('MSE: ', MSE)
    # print('PSNR: ', PSNR)


# print('sumMSE: ',sumMSE)
# print('sumPSNR: ', sumPSNR)
# print('sumSSIM: ', sumSSIM)
# print('count: ', count)


# Get the Average value 
mse_value = sumMSE/count
psnr_value = sumPSNR/count
ssim_value = sumSSIM/count


print("MSE for testing data :", format(mse_value, ".4f"))
print("PSNR for testing data :", format(psnr_value, ".4f"), "dB")
print("SSIM for testing data :", format(ssim_value, ".4f"))

MSE for testing data : 3951.7662
PSNR for testing data : 12.9747 dB
SSIM for testing data : 0.6707
